In [237]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

Data = pd.read_csv('seed.txt',header=None, sep="\s+", usecols=[0,1,2,3,4,5,6,7], names=['f1','f2','f3','f4','f5','f6','f7', 'label'])


In [238]:

class PreprocessData:
    def NomalizeData(Data):
        lenght = len(Data)
        normalizedData = []
        for i in range (lenght):
            normalizedData.append(float((Data[i] - min(Data) ) / ( max(Data) - min(Data)) ))
            
        return normalizedData
    
    def TestAndTrain(Data , PercentageOfTrainData):
        PercentageOfTrainData = float(PercentageOfTrainData / 100)
        Train_DataFrame = Data.sample(frac=PercentageOfTrainData)
        Test_DataFrame =Data.drop(Train_DataFrame.index)
        
        return Train_DataFrame , Test_DataFrame

In [239]:
#devide data to test and train 
Dataset = PreprocessData.TestAndTrain(Data , 80)

TrainData = Dataset[0]
TestData = Dataset[1]
X = TrainData[['f1','f2','f3','f4','f5','f6','f7']].to_numpy()
Xnonbias = X
X= np.c_[np.ones((len(X) , 1)) , X]
Y = TrainData['label'].to_numpy()

xtest =  TestData[['f1','f2','f3','f4','f5','f6','f7']].to_numpy()
ytest =  TestData['label'].to_numpy()


X.shape

(168, 8)

In [240]:
TrainData1 = TrainData[TrainData.label == 1]
TrainData2 = TrainData[TrainData.label == 2]
TrainData3 = TrainData[TrainData.label == 3]

In [256]:
#defining functions 
def sigmoid(z):
    return 1.0/(1+ np.exp(-1*z))

def hypotesis( X , theta):
    z = np.dot(X , theta)
    yHat = sigmoid(z)
    return yHat
def error(X , theta , y ,classin):
    cost = 0
    for i , j in zip(X , y):
        x_bar = np.array(np.insert(i, 0, 1))
        y_hat = hypotesis(x_bar, theta)
        classin = 1.0 if j == classin else 0.0
        cost += classin * np.log(y_hat) + (1.0 - classin) * np.log(1 - y_hat)
        
    return cost   
    
def train( input_var, label, itrations , classin , alpharate ,theta):
    classofInterst = classin
    theta1 = theta
    for a in range(itrations):
        
        
        if a % 1000 == 0:
            print(f'iteration: {a}')
            print(f'cost: {error(input_var,  theta1, label , classofInterst)}')
            print(theta1)
            print('--------------------------------------------')
        
        
        for i , xy in enumerate(zip(input_var , label)):
            x_bar = np.array(np.insert(xy[0], 0, 1))
            y_hat = hypotesis(x_bar,theta)
            
            y_binary = 1.0 if xy[1] == classin else 0.0 
             
            gradient = (y_binary - y_hat) * x_bar
            theta += alpharate * gradient
            
            
            
            
            
        
    return theta
            
        

In [257]:
alpha = 1e-2
params_0 = np.zeros(Xnonbias.shape[1] + 1)

max_iter = 20000


a =\
train(Xnonbias/ 8 , Y , 10000 , 1 , alpha , params_0)


iteration: 0
cost: -116.448726334071
[0. 0. 0. 0. 0. 0. 0. 0.]
--------------------------------------------
iteration: 1000
cost: -91.84211386681518
[ 6.04266408 -2.53304061  3.4254251   1.111151    0.39220865  3.3362598
 -8.38495561 -9.6068238 ]
--------------------------------------------
iteration: 2000
cost: -96.76701238618818
[  7.61771859  -2.17773566   4.57077654   1.62696037   0.28228389
   5.49294884  -8.1076525  -17.47654619]
--------------------------------------------
iteration: 3000
cost: -100.80840009206554
[  8.62022439  -1.88131152   5.59794713   2.00290378   0.48406676
   7.12218087  -7.75822293 -24.04806116]
--------------------------------------------
iteration: 4000
cost: -104.91519679847997
[  9.31289273  -1.72439802   6.59071301   2.2875875    0.93710336
   8.37612427  -7.53524044 -29.62760064]
--------------------------------------------


KeyboardInterrupt: 

In [254]:
class LogisticRegression():
    """Class for training and using a model for logistic regression"""
    
    def set_values(self, initial_params, alpha=0.01, max_iter=5000, class_of_interest=0):
        """Set the values for initial params, step size, maximum iteration, and class of interest"""
        self.params = initial_params
        self.alpha = alpha
        self.max_iter = max_iter
        self.class_of_interest = class_of_interest
    
    @staticmethod
    def _sigmoid(x):
        """Sigmoide function"""
        
        return 1.0 / (1.0 + np.exp(-x))
    
    def predict(self, x_bar, params):
        """predict the probability of a class"""  

        return self._sigmoid(np.dot(params, x_bar))
        
    
    def _compute_cost(self, input_var, output_var, params):
        """Compute the log likelihood cost"""
        
        cost = 0
        for x, y in zip(input_var, output_var):
            x_bar = np.array(np.insert(x, 0, 1))
            y_hat = self.predict(x_bar, params)
            
            y_binary = 1.0 if y == self.class_of_interest else 0.0
            cost += y_binary * np.log(y_hat) + (1.0 - y_binary) * np.log(1 - y_hat)
            
            
        return cost
    
    def train(self, input_var, label, print_iter = 5000):
        """Train the model using batch gradient ascent"""
        
        iteration = 1
        while iteration < self.max_iter:
            if iteration % print_iter == 0:
                print(f'iteration: {iteration}')
                print(f'cost: {self._compute_cost(input_var, label, self.params)}')
                print(self.params)
                print('--------------------------------------------')
            
            for i, xy in enumerate(zip(input_var, label)):
                x_bar = np.array(np.insert(xy[0], 0, 1))
                y_hat = self.predict(x_bar, self.params)
                
                y_binary = 1.0 if xy[1] == self.class_of_interest else 0.0
                gradient = (y_binary - y_hat) * x_bar
                self.params += self.alpha * gradient
            
            iteration +=1
        
        return self.params

    def test(self, input_test, label_test):
        """Test the accuracy of the model using test data"""
        self.total_classifications = 0
        self.correct_classifications = 0
        
        for x,y in zip(input_test, label_test):
            self.total_classifications += 1
            x_bar = np.array(np.insert(x, 0, 1))
            y_hat = self.predict(x_bar, self.params)
            y_binary = 1.0 if y == self.class_of_interest else 0.0
            
            if y_hat >= 0.5 and  y_binary == 1:
                # correct classification of class_of_interest
                self.correct_classifications += 1
              
            if y_hat < 0.5 and  y_binary != 1:
                # correct classification of an other class
                self.correct_classifications += 1
                
        self.accuracy = self.correct_classifications / self.total_classifications
            
        return self.accuracy

In [255]:
# train a classifier for the ZERO digit
alpha = 1e-2
params_0 = np.zeros(Xnonbias.shape[1] + 1)

max_iter = 80000
digits_regression_model_0 = LogisticRegression()
digits_regression_model_0.set_values(params_0, alpha, max_iter, 1)


digits_regression_model_0.train(Xnonbias/ 8.0, Y, 1000)



iteration: 1000
cost: -64.70657761702192
[ 6.04047207 -2.5331983   3.42406167  1.11052161  0.39244032  3.33375357
 -8.38473967 -9.59819297]
--------------------------------------------
iteration: 2000
cost: -57.67574960368013
[  7.61651477  -2.17810767   4.56972581   1.62652741   0.28223599
   5.49108827  -8.10806311 -17.4693805 ]
--------------------------------------------
iteration: 3000
cost: -52.902171572617156
[  8.6193938   -1.88153192   5.59693779   2.00257822   0.48372539
   7.12075871  -7.75850325 -24.04203077]
--------------------------------------------


KeyboardInterrupt: 